In [ ]:
##Tools Creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun #queryrunner
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper #helps query runner

In [ ]:
##Used the inbuilt tools
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

In [ ]:
##Used the inbuilt tools
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

In [ ]:
tools = [wiki, arxiv]

In [ ]:
##Custom Total
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, embeddings)
retriever = vectordb.as_retriever()
result= retriever.invoke("What is LangSmith?")
print(result[0])

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever=retriever, name="langsmith-search", description="search anu information about LangSmith")
retriever_tool.name

In [ ]:
tools = [wiki, arxiv, retriever_tool]
tools


In [ ]:

from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=api_key, model_name="gemma2-9b-it")
chain = llm | parser
chain.invoke("What is LangSmith?")

In [ ]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)
agent

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input": "What is LangSmith?"})  # Use the correct key

In [ ]:
agent_executor.invoke({"input": "Who is narendra modi?"})  # Use the correct key